<a href="https://colab.research.google.com/github/UniVR-DH/ADHLab/blob/main/lecture04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Producing N-Grams and Building Inverted Indexes


<img src="https://drive.google.com/uc?export=view&id=1m_EMdnI5C826kgqK7r5vB4TXnB0-Wq7W" alt="Intestazione con loghi istituzionali" width="525"/>

| Docente      | Insegnamento | Anno Accademico    |
| :---        |    :----   |          ---: |
| Matteo Lissandrini      | Laboratorio Avanzato di Informatica Umanistica       | 2023/2024   |

## Usual install and basic imports

In [ ]:
%pip install wikipedia-api

In [ ]:
import gzip
import math
import string
import requests
import numpy as np
import regex as re
import matplotlib.pyplot as plt
from collections import Counter

## Declare Wikipedia API Object to download pages

In [ ]:
import wikipediaapi
wapi_text = wikipediaapi.Wikipedia('MyProjectName (name@studenti.univr.it)',
                                   'en',
                                   extract_format=wikipediaapi.ExtractFormat.WIKI)

punct_regex = re.compile('[{}]'.format(re.escape(string.punctuation))) # Regex matching any punctuation
space_regex = re.compile(' +') # Regex matching whitespace

In [ ]:
page_py = wapi_text.page('New York City')
print("Page - Exists: %s" % page_py.exists())
print(page_py.text[-140:])

page_tokens = space_regex.sub(' ', punct_regex.sub(' ', page_py.text.lower())).strip().split()
print(len(page_tokens))
# Create the bag of words
page_bow = Counter(page_tokens)

print(len(page_bow))

Page - Exists: True
 145,000 NYC photographs at the Museum of the City of New York
"The New New York Skyline (interactive)". National Geographic. November 2015.
16157
3452


### Goal:  build an inverted index on a set of bigrams



In [ ]:
# request the raw text of Alice in Wonderland
r = requests.get(r'https://ia801604.us.archive.org/6/items/alicesadventures19033gut/19033.txt')
alice = r.text

alice_pages = alice.split("\n\r\n\r\n\r")
alice_pages = [ space_regex.sub(' ', page).strip() for page in alice_pages ]
alice_pages = [ page for page in alice_pages if page != "" ]
print(len(alice_pages))

print(" ".join(alice_pages[7].splitlines() ))


19
I--DOWN THE RABBIT-HOLE   Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do. Once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, "and what is the use of a book," thought Alice, "without pictures or conversations?"  So she was considering in her own mind (as well as she could, for the day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when suddenly a White Rabbit with pink eyes ran close by her.  There was nothing so very remarkable in that, nor did Alice think it so very much out of the way to hear the Rabbit say to itself, "Oh dear! Oh dear! I shall be too late!" But when the Rabbit actually took a watch out of its waistcoat-pocket and looked at it and then hurried on, Alice started to her feet, for it flashed across her mind that she had never before seen a rabbit w

In [ ]:
bigrams = set()
inverted_index = {}

for page_position, page in enumerate(alice_pages):
  target_page_oneline = space_regex.sub(' ', " ".join(page.splitlines()))
  page_word_list = target_page_oneline.split(' ')
  print(len(page_word_list))

  for position, word in enumerate(page_word_list):
    if position+1 <= len(page_word_list)-1:

      word2 = page_word_list[position+1]
      bigram = ( word , word2 )
      #print(bigram)
      bigrams.add(bigram)

      if bigram in inverted_index:
        inverted_index[bigram].append( page_position)
      else:
        inverted_index[bigram] = [page_position]


print('===')
print(len(bigrams))
print(len(inverted_index))
print(inverted_index[('Alice', 'was')])

89
14
11
4
7
10
5
1344
1146
1065
1339
1065
917
567
1133
564
551
231
2705
===
9229
9229
[7, 7, 9, 11, 12, 12, 14, 14]


In [ ]:
r = requests.get(r'https://ia600906.us.archive.org/29/items/aesopsfablesanew11339gut/11339.txt')
fables = r.text

fables_pages = fables.split("\n\r\n\r\n\r")
fables_pages = [ space_regex.sub(' ', page).strip() for page in fables_pages ]
fables_pages = [ page for page in fables_pages if page != "" ]
print(len(fables_pages))
print(" ".join(fables_pages[3].splitlines()))

294
INTRODUCTION   _AEsop embodies an epigram not uncommon in human history; his fame is all the more deserved because he never deserved it. The firm foundations of common sense, the shrewd shots at uncommon sense, that characterise all the Fables, belong not him but to humanity. In the earliest human history whatever is authentic is universal: and whatever is universal is anonymous. In such cases there is always some central man who had first the trouble of collecting them, and afterwards the fame of creating them. He had the fame; and, on the whole, he earned the fame. There must have been something great and human, something of the human future and the human past, in such a man: even if he only used it to rob the past or deceive the future. The story of Arthur may have been really connected with the most fighting Christianity of falling Rome or with the most heathen traditions hidden in the hills of Wales. But the word "Mappe" or "Malory" will always mean King Arthur; even though we

In [ ]:
### TODO Repeat the same for Aesop's fables

In [ ]:
### Analyze bigram frequencies